In [27]:
import pandas as pd
import re
from shapely.geometry import LineString, MultiPoint
from shapely.ops import linemerge



In [25]:
df = pd.read_csv('linhas_vlt_rj.csv')  # Substitua pelo caminho do seu arquivo

In [26]:

# Dicionário para armazenar as tabelas separadas por 'nome'
tabelas_por_nome = {}

# Processar cada linha do DataFrame original
for _, row in df.iterrows():
    nome = row['nome']
    geometry = row['geometry']
    
    # Extrair todos os segmentos LINESTRING (cada parte entre parênteses separados por vírgula)
    segmentos = re.findall(r'\(([^)]+)\)', geometry)
    
    # Criar um DataFrame temporário para os segmentos desta linha
    segmentos_df = pd.DataFrame({
        'id': [f'linestring_{i+1}' for i in range(len(segmentos))],
        'geometry': [f'LINESTRING({seg})' for seg in segmentos],
        'objectid': row['objectid'],
        'em_operação': row['em_operação'],
        'cod_linha': row['cod_linha'],
        'st_length(shape)': row['st_length(shape)']
    })
    
    # Adicionar ao DataFrame correspondente ao 'nome' (criar se não existir)
    if nome not in tabelas_por_nome:
        tabelas_por_nome[nome] = segmentos_df
    else:
        tabelas_por_nome[nome] = pd.concat([tabelas_por_nome[nome], segmentos_df], ignore_index=True)

# Exemplo: Salvar cada tabela em um arquivo CSV separado
for nome, tabela in tabelas_por_nome.items():
    nome_arquivo = f"{nome.replace(' ', '_').replace('-', '_')}.csv"  # Ex: "Linha_1_Azul.csv"
    tabela.to_csv(nome_arquivo, index=False)
    print(f"Arquivo salvo: {nome_arquivo}")


Arquivo salvo: Linha_1___Azul.csv
Arquivo salvo: Linha_2___Verde.csv
Arquivo salvo: Linha_3___Amarela.csv
Arquivo salvo: Linha_4___Laranja.csv


In [28]:
df = pd.read_csv('Linha_1___Azul.csv')  

# Função para extrair pontos de uma LINESTRING
def extract_points(linestring):
    # Extrai os pares de coordenadas usando regex
    points = re.findall(r'(-?\d+\.\d+)\s+(-?\d+\.\d+)', linestring)
    return [(float(lon), float(lat)) for lon, lat in points]

# Lista para armazenar todos os pontos únicos
all_points = []

# Processar cada LINESTRING no DataFrame
for _, row in df.iterrows():
    geometry = row['geometry']
    points = extract_points(geometry)
    all_points.extend(points)

# Remover pontos duplicados (usando um conjunto)
unique_points = list(dict.fromkeys(all_points))  # Mantém a ordem de inserção

# --- Opção 1: Ordenar por proximidade (usando Shapely) ---
if unique_points:
    # Criar um LineString e ordenar os pontos pela distância ao longo da linha
    line = LineString(unique_points)
    ordered_points = [line.coords[0]]  # Começa com o primeiro ponto
    remaining_points = set(unique_points[1:])
    
    while remaining_points:
        last_point = ordered_points[-1]
        # Encontra o ponto mais próximo do último ponto adicionado
        closest_point = min(remaining_points, key=lambda p: LineString([last_point, p]).length)
        ordered_points.append(closest_point)
        remaining_points.remove(closest_point)
    
    # Criar a LINESTRING ordenada e sem duplicatas
    merged_linestring = f"LINESTRING({', '.join([f'{lon} {lat}' for lon, lat in ordered_points])})"
else:
    merged_linestring = "LINESTRING EMPTY"


# Criar um novo DataFrame com o resultado
merged_df = pd.DataFrame({
    'id': ['merged_linestring'],
    'geometry': [merged_linestring],
    'objectid': [df['objectid'].iloc[0]],  # Pega o primeiro objectid (ajuste conforme necessário)
    'em_operação': [df['em_operação'].iloc[0]],
    'cod_linha': [df['cod_linha'].iloc[0]],
    'st_length(shape)': [LineString(ordered_points).length if unique_points else 0]
})

# Salvar o resultado em um novo arquivo
merged_df.to_csv('linestring_Linha_1_Azul.csv', index=False)



In [30]:
df = pd.read_csv('Linha_2___Verde.csv')  

# Função para extrair pontos de uma LINESTRING
def extract_points(linestring):
    # Extrai os pares de coordenadas usando regex
    points = re.findall(r'(-?\d+\.\d+)\s+(-?\d+\.\d+)', linestring)
    return [(float(lon), float(lat)) for lon, lat in points]

# Lista para armazenar todos os pontos únicos
all_points = []

# Processar cada LINESTRING no DataFrame
for _, row in df.iterrows():
    geometry = row['geometry']
    points = extract_points(geometry)
    all_points.extend(points)

# Remover pontos duplicados (usando um conjunto)
unique_points = list(dict.fromkeys(all_points))  # Mantém a ordem de inserção

# --- Opção 1: Ordenar por proximidade (usando Shapely) ---
if unique_points:
    # Criar um LineString e ordenar os pontos pela distância ao longo da linha
    line = LineString(unique_points)
    ordered_points = [line.coords[0]]  # Começa com o primeiro ponto
    remaining_points = set(unique_points[1:])
    
    while remaining_points:
        last_point = ordered_points[-1]
        # Encontra o ponto mais próximo do último ponto adicionado
        closest_point = min(remaining_points, key=lambda p: LineString([last_point, p]).length)
        ordered_points.append(closest_point)
        remaining_points.remove(closest_point)
    
    # Criar a LINESTRING ordenada e sem duplicatas
    merged_linestring = f"LINESTRING({', '.join([f'{lon} {lat}' for lon, lat in ordered_points])})"
else:
    merged_linestring = "LINESTRING EMPTY"


# Criar um novo DataFrame com o resultado
merged_df = pd.DataFrame({
    'id': ['merged_linestring'],
    'geometry': [merged_linestring],
    'objectid': [df['objectid'].iloc[0]],  # Pega o primeiro objectid (ajuste conforme necessário)
    'em_operação': [df['em_operação'].iloc[0]],
    'cod_linha': [df['cod_linha'].iloc[0]],
    'st_length(shape)': [LineString(ordered_points).length if unique_points else 0]
})

# Salvar o resultado em um novo arquivo
merged_df.to_csv('linestring_Linha_2_Verde.csv', index=False)



In [31]:
df = pd.read_csv('Linha_3___Amarela.csv')  

# Função para extrair pontos de uma LINESTRING
def extract_points(linestring):
    # Extrai os pares de coordenadas usando regex
    points = re.findall(r'(-?\d+\.\d+)\s+(-?\d+\.\d+)', linestring)
    return [(float(lon), float(lat)) for lon, lat in points]

# Lista para armazenar todos os pontos únicos
all_points = []

# Processar cada LINESTRING no DataFrame
for _, row in df.iterrows():
    geometry = row['geometry']
    points = extract_points(geometry)
    all_points.extend(points)

# Remover pontos duplicados (usando um conjunto)
unique_points = list(dict.fromkeys(all_points))  # Mantém a ordem de inserção

# --- Opção 1: Ordenar por proximidade (usando Shapely) ---
if unique_points:
    # Criar um LineString e ordenar os pontos pela distância ao longo da linha
    line = LineString(unique_points)
    ordered_points = [line.coords[0]]  # Começa com o primeiro ponto
    remaining_points = set(unique_points[1:])
    
    while remaining_points:
        last_point = ordered_points[-1]
        # Encontra o ponto mais próximo do último ponto adicionado
        closest_point = min(remaining_points, key=lambda p: LineString([last_point, p]).length)
        ordered_points.append(closest_point)
        remaining_points.remove(closest_point)
    
    # Criar a LINESTRING ordenada e sem duplicatas
    merged_linestring = f"LINESTRING({', '.join([f'{lon} {lat}' for lon, lat in ordered_points])})"
else:
    merged_linestring = "LINESTRING EMPTY"


# Criar um novo DataFrame com o resultado
merged_df = pd.DataFrame({
    'id': ['merged_linestring'],
    'geometry': [merged_linestring],
    'objectid': [df['objectid'].iloc[0]],  # Pega o primeiro objectid (ajuste conforme necessário)
    'em_operação': [df['em_operação'].iloc[0]],
    'cod_linha': [df['cod_linha'].iloc[0]],
    'st_length(shape)': [LineString(ordered_points).length if unique_points else 0]
})

# Salvar o resultado em um novo arquivo
merged_df.to_csv('linestring_Linha_3_Amarela.csv', index=False)



In [32]:
df = pd.read_csv('Linha_4___Laranja.csv')  

# Função para extrair pontos de uma LINESTRING
def extract_points(linestring):
    # Extrai os pares de coordenadas usando regex
    points = re.findall(r'(-?\d+\.\d+)\s+(-?\d+\.\d+)', linestring)
    return [(float(lon), float(lat)) for lon, lat in points]

# Lista para armazenar todos os pontos únicos
all_points = []

# Processar cada LINESTRING no DataFrame
for _, row in df.iterrows():
    geometry = row['geometry']
    points = extract_points(geometry)
    all_points.extend(points)

# Remover pontos duplicados (usando um conjunto)
unique_points = list(dict.fromkeys(all_points))  # Mantém a ordem de inserção

# --- Opção 1: Ordenar por proximidade (usando Shapely) ---
if unique_points:
    # Criar um LineString e ordenar os pontos pela distância ao longo da linha
    line = LineString(unique_points)
    ordered_points = [line.coords[0]]  # Começa com o primeiro ponto
    remaining_points = set(unique_points[1:])
    
    while remaining_points:
        last_point = ordered_points[-1]
        # Encontra o ponto mais próximo do último ponto adicionado
        closest_point = min(remaining_points, key=lambda p: LineString([last_point, p]).length)
        ordered_points.append(closest_point)
        remaining_points.remove(closest_point)
    
    # Criar a LINESTRING ordenada e sem duplicatas
    merged_linestring = f"LINESTRING({', '.join([f'{lon} {lat}' for lon, lat in ordered_points])})"
else:
    merged_linestring = "LINESTRING EMPTY"


# Criar um novo DataFrame com o resultado
merged_df = pd.DataFrame({
    'id': ['merged_linestring'],
    'geometry': [merged_linestring],
    'objectid': [df['objectid'].iloc[0]],  # Pega o primeiro objectid (ajuste conforme necessário)
    'em_operação': [df['em_operação'].iloc[0]],
    'cod_linha': [df['cod_linha'].iloc[0]],
    'st_length(shape)': [LineString(ordered_points).length if unique_points else 0]
})

# Salvar o resultado em um novo arquivo
merged_df.to_csv('linestring_Linha_4_Laranja.csv', index=False)

